In [1]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
from PIL import Image
import requests
import torch

In [2]:
checkpoint = "openai/clip-vit-large-patch14"
model = AutoModelForZeroShotImageClassification.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [3]:
url = ["https://thumbs.dreamstime.com/z/girl-crying-over-makeup-sad-person-concept-isolated-girl-crying-over-makeup-sad-person-concept-isolated-white-174239618.jpg",
       "https://www.allprodad.com/wp-content/uploads/2021/03/05-12-21-happy-people.jpg"]
sad_img = Image.open(requests.get(url[0], stream=True).raw)
happy_img = Image.open(requests.get(url[1], stream=True).raw)
images = [sad_img, happy_img]

images

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x1034>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1400x880>]

In [4]:
candidate_labels = ["happy", "sad", "angry", "shocked", "neutral"]
inputs = processor(images=images, text=candidate_labels, return_tensors="pt", padding=True)

In [5]:
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

result = [
    {"score": score, "label": candidate_label}
    for score, candidate_label in sorted(zip(probs, candidate_labels), key=lambda x: -x[0])
]

In [8]:
result

[{'score': 0.91687083, 'label': 'sad'},
 {'score': 0.044058256, 'label': 'neutral'},
 {'score': 0.028159017, 'label': 'angry'},
 {'score': 0.007031957, 'label': 'shocked'},
 {'score': 0.003879777, 'label': 'happy'}]